## Debugging the radar scripts line by line

In [1]:
import acconeer.exptool.a121 as a121

In [2]:
print(dir(a121))

['Client', 'ClientError', 'ClientInfo', 'Criticality', 'ExampleArgumentParser', 'H5Record', 'H5Recorder', 'IdleState', 'InMemoryRecord', 'Metadata', 'MockInfo', 'PRF', 'PersistentRecord', 'Profile', 'Record', 'RecordError', 'Recorder', 'Result', 'SDK_VERSION', 'SensorCalibration', 'SensorConfig', 'SensorInfo', 'SerialInfo', 'ServerError', 'ServerInfo', 'SessionConfig', 'SocketInfo', 'StackedResults', 'SubsweepConfig', 'USBInfo', 'ValidationError', 'ValidationResult', 'ValidationWarning', '_H5PY_STR_DTYPE', '_ReplayingClient', '_SensorPerformanceCalc', '_SessionPerformanceCalc', '_StopReplay', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_core', '_core_ext', '_h5_utils', '_perf_calc', 'complex_array_to_int16_complex', 'get_client_args', 'get_point_overhead_duration', 'get_sample_duration', 'int16_complex_array_to_complex', 'iterate_extended_structure', 'iterate_extended_structure_values', 'load_record', 'open_reco

In [2]:
print(dir(a121))

['Client', 'ClientError', 'ClientInfo', 'Criticality', 'ExampleArgumentParser', 'H5Record', 'H5Recorder', 'IdleState', 'InMemoryRecord', 'Metadata', 'MockInfo', 'PRF', 'PersistentRecord', 'Profile', 'Record', 'RecordError', 'Recorder', 'Result', 'SDK_VERSION', 'SensorCalibration', 'SensorConfig', 'SensorInfo', 'SerialInfo', 'ServerError', 'ServerInfo', 'SessionConfig', 'SocketInfo', 'StackedResults', 'SubsweepConfig', 'USBInfo', 'ValidationError', 'ValidationResult', 'ValidationWarning', '_H5PY_STR_DTYPE', '_ReplayingClient', '_SensorPerformanceCalc', '_SessionPerformanceCalc', '_StopReplay', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_cli', '_core', '_core_ext', '_h5_utils', '_perf_calc', 'complex_array_to_int16_complex', 'get_client_args', 'get_point_overhead_duration', 'get_sample_duration', 'int16_complex_array_to_complex', 'iterate_extended_structure', 'iterate_extended_structure_values', 'load_record', 'o

In [5]:
from acconeer.exptool.a121 import Client
from acconeer.exptool.a121.algo.breathing import RefApp
from acconeer.exptool.a121.algo.breathing._ref_app import (BreathingProcessorConfig, RefAppConfig, get_sensor_config)
from acconeer.exptool.a121.algo.presence import ProcessorConfig as PresenceProcessorConfig

In [8]:
client = Client.open()#serial_port="/dev/spidev0.1")

LinkError: Cannot auto detect:
USB: No USB devices detected. Cannot auto detect.
Serial: No serial devices detected. Cannot auto detect.

In [ ]:
from acconeer.exptool.a121.algo.breathing

In [1]:
from __future__ import annotations

import acconeer.exptool as et
from acconeer.exptool import a121
from acconeer.exptool.a121.algo.breathing import RefApp
from acconeer.exptool.a121.algo.breathing._ref_app import (
    BreathingProcessorConfig,
    RefAppConfig,
    get_sensor_config,
)
from acconeer.exptool.a121.algo.presence import ProcessorConfig as PresenceProcessorConfig


In [2]:
#args = a121.ExampleArgumentParser().parse_args()
#et.utils.config_logging(args)

# Setup the configurations
# Detailed at https://docs.acconeer.com/en/latest/exploration_tool/algo/a121/ref_apps/breathing.html

# Sensor selections
sensor = 1

# Ref App Configurations
breathing_processor_config = BreathingProcessorConfig(
    lowest_breathing_rate=6,
    highest_breathing_rate=60,
    time_series_length_s=20,
)

# Presence Configurations
presence_config = PresenceProcessorConfig(
    intra_detection_threshold=4,
    intra_frame_time_const=0.15,
    inter_frame_fast_cutoff=20,
    inter_frame_slow_cutoff=0.2,
    inter_frame_deviation_time_const=0.5,
)

# Breathing Configurations
ref_app_config = RefAppConfig(
    use_presence_processor=True,
    num_distances_to_analyze=3,
    hwaas=64,
    distance_determination_duration=5,
    breathing_config=breathing_processor_config,
    presence_config=presence_config,
)

In [3]:
# Preparation for client
sensor_config = get_sensor_config(ref_app_config=ref_app_config)
sensor_config

SensorConfig(subsweeps=[SubsweepConfig(start_point=119, num_points=21, step_length=24, profile=<Profile.PROFILE_3: 3>, hwaas=64, receiver_gain=16, enable_tx=True, enable_loopback=False, phase_enhancement=False, _prf=<PRF.PRF_15_6_MHz: (15600000, 5.1, 9.6)>)], sweeps_per_frame=16, sweep_rate=None, frame_rate=20.0, continuous_sweep_mode=False, double_buffering=False, inter_frame_idle_state=IdleState.DEEP_SLEEP, inter_sweep_idle_state=IdleState.READY)

In [4]:
client = a121.Client.open(ip_address="192.168.50.175")
print(client.server_info)

ServerInfo:
  rss_version ............ a121-v1.5.0
  sensor_count ........... 5
  ticks_per_second ....... 1000000
  hardware_name .......... linux
  max_baudrate ........... None
  sensor_infos:
    SensorInfo @ slot 1:
      connected .............. True
      serial ................. None
    SensorInfo @ slot 2:
      connected .............. False
      serial ................. None
    SensorInfo @ slot 3:
      connected .............. False
      serial ................. None
    SensorInfo @ slot 4:
      connected .............. False
      serial ................. None
    SensorInfo @ slot 5:
      connected .............. False
      serial ................. None


In [11]:
client.close()

In [ ]:

client.setup_session(sensor_config)

# Preparation for reference application processor
ref_app = RefApp(client=client, sensor_id=sensor, ref_app_config=ref_app_config)
ref_app.start()

In [ ]:


def main():
    args = a121.ExampleArgumentParser().parse_args()
    et.utils.config_logging(args)

    # Setup the configurations
    # Detailed at https://docs.acconeer.com/en/latest/exploration_tool/algo/a121/ref_apps/breathing.html

    # Sensor selections
    sensor = 1

    # Ref App Configurations
    breathing_processor_config = BreathingProcessorConfig(
        lowest_breathing_rate=6,
        highest_breathing_rate=60,
        time_series_length_s=20,
    )

    # Presence Configurations
    presence_config = PresenceProcessorConfig(
        intra_detection_threshold=4,
        intra_frame_time_const=0.15,
        inter_frame_fast_cutoff=20,
        inter_frame_slow_cutoff=0.2,
        inter_frame_deviation_time_const=0.5,
    )

    # Breathing Configurations
    ref_app_config = RefAppConfig(
        use_presence_processor=True,
        num_distances_to_analyze=3,
        distance_determination_duration=5,
        breathing_config=breathing_processor_config,
        presence_config=presence_config,
    )

    # End setup configurations

    # Preparation for client
    sensor_config = get_sensor_config(ref_app_config=ref_app_config)
    client = a121.Client.open(**a121.get_client_args(args))
    client.setup_session(sensor_config)

    # Preparation for reference application processor
    ref_app = RefApp(client=client, sensor_id=sensor, ref_app_config=ref_app_config)
    ref_app.start()

    interrupt_handler = et.utils.ExampleInterruptHandler()
    print("Press Ctrl-C to end session")

    while not interrupt_handler.got_signal:
        processed_data = ref_app.get_next()
        try:
            print("Breathing result " + str(processed_data.breathing_result))
        except et.PGProccessDiedException:
            break

    ref_app.stop()
    print("Disconnecting...")
    client.close()


if __name__ == "__main__":
    main()